<a href="https://colab.research.google.com/github/JawadArnob/Pattern-Lab/blob/main/Assign3_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/11th trimester/pattern lab

/content/gdrive/MyDrive/11th trimester/pattern lab


In [3]:
import numpy as np
import pickle
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [4]:
Xtrain = np.load(f'./final datas/Xtrain.npy')
Ytrain = np.load(f'./final datas/Ytrain.npy')
Xtest = np.load(f'./final datas/Xtest.npy')
Ytest = np.load(f'./final datas/Ytest.npy')

with open('index_dict.pickle', 'rb') as handle:
    index_dict = pickle.load(handle)
index_dict

{'(CKSNAP1)': [820, 851],
 '(CKSNAP3)': [852, 915],
 '(CKSNAP5)': [916, 1011],
 '(CKSNAP7)': [1012, 1139],
 '(CKSNAP9)': [1140, 1299],
 '(DAC7)': [1300, 1341],
 '(EIIP)': [1342, 1382],
 '(ENAC10)': [1531, 1658],
 '(ENAC5)': [1383, 1530],
 '(NCP)': [3023, 3145],
 '(PseEIIP)': [3146, 3209],
 '(TAC7)': [3210, 3223],
 '(binary)': [0, 819],
 '(kmer1)': [1659, 1662],
 '(kmer2)': [1663, 1678],
 '(kmer3)': [1679, 1742],
 '(kmer4)': [1743, 1998],
 '(kmer5)': [1999, 3022]}

In [11]:
random.seed(13)

features ={1 : '(CKSNAP1)', 2 : '(CKSNAP3)', 3 : '(CKSNAP5)', 4 : '(CKSNAP7)', 5 : '(CKSNAP9)', 6 : '(DAC7)', 7 : '(EIIP)', 8 : '(ENAC5)', 9 : '(ENAC10)', 10 : '(kmer1)', 11 : '(kmer2)', 12 : '(kmer3)', 13 : '(kmer4)', 14 : '(kmer5)', 15 : '(NCP)', 16 : '(PseEIIP)', 17 : '(TAC7)', 18 : '(binary)'}


subgrp1 = [features[random.randint(1,18)] for i in range(5)]
subgrp2 = [features[random.randint(1,18)] for i in range(5)]
subgrp3 = [features[random.randint(1,18)] for i in range(5)]
subgrp4 = [features[random.randint(1,18)] for i in range(5)]
subgrp5 = [features[random.randint(1,18)] for i in range(5)]




In [12]:
X1 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp1], axis = 1)
X2 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp2], axis = 1)
X3 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp3], axis = 1)
X4 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp4], axis = 1)
X5 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp5], axis = 1)

Xs = [X1, X2, X3, X4, X5]

In [9]:
subgroups = [subgrp1, subgrp2, subgrp3, subgrp4, subgrp5]
for group in subgroups:
  print(group)

['(ENAC10)', '(Kmer1)', '(DAC7)', '(ENAC5)', '(CKSNAP9)']
['(ENAC5)', '(DAC7)', '(CKSNAP9)', '(CKSNAP5)', '(binary)']
['(EIIP)', '(Kmer1)', '(CKSNAP1)', '(Kmer5)', '(CKSNAP9)']
['(CKSNAP1)', '(ENAC10)', '(CKSNAP9)', '(CKSNAP5)', '(ENAC10)']
['(NCP)', '(Kmer5)', '(CKSNAP9)', '(ENAC10)', '(Kmer3)']


In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from numpy import mean

kf = KFold(n_splits=5, shuffle = True)

In [15]:
lg = LogisticRegression(max_iter=500)
for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in logistic regression \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in logistic regression \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in logistic regression \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in logistic regression \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in logistic regression 
 train -> 0.7969936130958017  test -> 0.7670612524799132
subgroup1 sensitivity in logistic regression 
 train -> 0.797096465178881  test -> 0.7677965086567575
subgroup1 specificity in logistic regression 
 train -> 0.7968418193388324  test -> 0.7655371292168445
subgroup1 mcc in logistic regression 
 train -> 0.5939701707304732  test -> 0.5336304870429692
subgroup2 accuracy in logistic regression 
 train -> 0.8108069069449817  test -> 0.7759992078946499
subgroup2 sensitivity in logistic regression 
 train -> 0.8069504425419968  test -> 0.7803915367167571
subgroup2 specificity in logistic regression 
 train -> 0.8145709092503921  test -> 0.7726418183205054
subgroup2 mcc in logistic regression 
 train -> 0.6215778571426094  test -> 0.5526594718079532
subgroup3 accuracy in logistic regression 
 train -> 0.7363213935497244  test -> 0.7080108987828714
subgroup3 sensitivity in logistic regression 
 train -> 0.7559180763124465  test -> 0.7230384984127

xg boost

In [17]:
xgbst = XGBClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    xgbst.fit(X_train, y_train)
    y_pred = xgbst.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    xgbst.fit(X_train, y_train)
    y_pred = xgbst.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in XG boost \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in XG boost \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in XG boost \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in XG boost \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in XG boost 
 train -> 0.8422268209623986  test -> 0.7267137529291396
subgroup1 sensitivity in XG boost 
 train -> 0.8508183560264723  test -> 0.7347144872818256
subgroup1 specificity in XG boost 
 train -> 0.8335185217396696  test -> 0.7196967282765124
subgroup1 mcc in XG boost 
 train -> 0.6845201143922163  test -> 0.45410627790690183
subgroup2 accuracy in XG boost 
 train -> 0.867483648704449  test -> 0.7730291795475465
subgroup2 sensitivity in XG boost 
 train -> 0.8736433455218648  test -> 0.7731908454313859
subgroup2 specificity in XG boost 
 train -> 0.861335381357612  test -> 0.7731072852079992
subgroup2 mcc in XG boost 
 train -> 0.735030319763113  test -> 0.5461586088235059
subgroup3 accuracy in XG boost 
 train -> 0.8552277259094809  test -> 0.7602942524689118
subgroup3 sensitivity in XG boost 
 train -> 0.8607633736924167  test -> 0.7611685325036992
subgroup3 specificity in XG boost 
 train -> 0.8496674769108333  test -> 0.759529015979815
subgroup3 mcc in

ada boost


In [18]:
ab = AdaBoostClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    ab.fit(X_train, y_train)
    y_pred = ab.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    ab.fit(X_train, y_train)
    y_pred = ab.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in Ada boost \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in Ada boost \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in Ada boost \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in Ada boost \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in Ada boost 
 train -> 0.7557557683784848  test -> 0.7004345577961868
subgroup1 sensitivity in Ada boost 
 train -> 0.7551234948835692  test -> 0.7040119757152761
subgroup1 specificity in Ada boost 
 train -> 0.7563502770567675  test -> 0.6974976369836382
subgroup1 mcc in Ada boost 
 train -> 0.5114899246381247  test -> 0.40155771229577264
subgroup2 accuracy in Ada boost 
 train -> 0.7888004831275632  test -> 0.753794954728979
subgroup2 sensitivity in Ada boost 
 train -> 0.7841252199231638  test -> 0.7531717031944473
subgroup2 specificity in Ada boost 
 train -> 0.7932706394660319  test -> 0.7545069575165145
subgroup2 mcc in Ada boost 
 train -> 0.5776368944063591  test -> 0.5075702816106136
subgroup3 accuracy in Ada boost 
 train -> 0.7834501296662838  test -> 0.7397031805230097
subgroup3 sensitivity in Ada boost 
 train -> 0.7823363461061155  test -> 0.736686086087331
subgroup3 specificity in Ada boost 
 train -> 0.7845196386940338  test -> 0.7427435574624485
sub

random forest

In [19]:
rf = RandomForestClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in Random forest \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in Random forest \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in Random forest \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in Random forest \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in Random forest 
 train -> 1.0  test -> 0.6912241328096637
subgroup1 sensitivity in Random forest 
 train -> 1.0  test -> 0.6963330041591892
subgroup1 specificity in Random forest 
 train -> 1.0  test -> 0.6870122787993362
subgroup1 mcc in Random forest 
 train -> 1.0  test -> 0.3833179581183671
subgroup2 accuracy in Random forest 
 train -> 1.0  test -> 0.7082896025171348
subgroup2 sensitivity in Random forest 
 train -> 1.0  test -> 0.7074716296131555
subgroup2 specificity in Random forest 
 train -> 1.0  test -> 0.7092020394403412
subgroup2 mcc in Random forest 
 train -> 1.0  test -> 0.4163971006870713
subgroup3 accuracy in Random forest 
 train -> 1.0  test -> 0.7007106945223714
subgroup3 sensitivity in Random forest 
 train -> 1.0  test -> 0.7080405118257072
subgroup3 specificity in Random forest 
 train -> 1.0  test -> 0.6937011306048664
subgroup3 mcc in Random forest 
 train -> 1.0  test -> 0.40158670270068353
subgroup4 accuracy in Random forest 
 train -> 1

svc

In [20]:
svc = SVC()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in svc \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in svc \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in svc \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in svc \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

subgroup1 accuracy in svc 
 train -> 0.8454766746278471  test -> 0.7489147056558523
subgroup1 sensitivity in svc 
 train -> 0.8574883747568169  test -> 0.7592259962623574
subgroup1 specificity in svc 
 train -> 0.8333588428815581  test -> 0.7385889919054335
subgroup1 mcc in svc 
 train -> 0.6911432247800637  test -> 0.4978337917733585
subgroup2 accuracy in svc 
 train -> 0.8976162313619763  test -> 0.7651664338023624
subgroup2 sensitivity in svc 
 train -> 0.9044022437141285  test -> 0.7675616382190162
subgroup2 specificity in svc 
 train -> 0.890832648262105  test -> 0.7633029220862055
subgroup2 mcc in svc 
 train -> 0.79530496137291  test -> 0.5308775517347225
subgroup3 accuracy in svc 
 train -> 0.7688925185615034  test -> 0.7215665350158237
subgroup3 sensitivity in svc 
 train -> 0.7873645677187115  test -> 0.7403525600464702
subgroup3 specificity in svc 
 train -> 0.7503877737829511  test -> 0.7034180181083414
subgroup3 mcc in svc 
 train -> 0.5381359355911435  test -> 0.444187247

indep test lg

In [21]:
lg = LogisticRegression(max_iter=500)

acc = []
sensitivity = []
specificity = []
mcc = []
lg.fit(Xtrain, Ytrain)
y_pred = lg.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.7992424242424242 
 sensitivity -> 0.7727272727272727 
 specificity -> 0.8257575757575758 
 mcc -> 0.5993281610305964


inde test xg

In [22]:
xgbst = XGBClassifier()
acc = []
sensitivity = []
specificity = []
mcc = []
xgbst.fit(Xtrain, Ytrain)
y_pred = xgbst.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.7916666666666666 
 sensitivity -> 0.803030303030303 
 specificity -> 0.7803030303030303 
 mcc -> 0.5834840459914051


indep ada

In [23]:
ab = AdaBoostClassifier()

acc = []
sensitivity = []
specificity = []
mcc = []
ab.fit(Xtrain, Ytrain)
y_pred = ab.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.7537878787878788 
 sensitivity -> 0.7348484848484849 
 specificity -> 0.7727272727272727 
 mcc -> 0.5079402853927962


indep rand forest

In [24]:

rf = RandomForestClassifier()

acc = []
sensitivity = []
specificity = []
mcc = []
rf.fit(Xtrain, Ytrain)
y_pred = rf.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.7045454545454546 
 sensitivity -> 0.7045454545454546 
 specificity -> 0.7045454545454546 
 mcc -> 0.4090909090909091


indep svc

In [25]:
svc = SVC()
acc = []
sensitivity = []
specificity = []
mcc = []
svc.fit(Xtrain, Ytrain)
y_pred = svc.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

acc -> 0.803030303030303 
 sensitivity -> 0.803030303030303 
 specificity -> 0.803030303030303 
 mcc -> 0.6060606060606061
